In [2]:
import time
import hashlib
import json
import random
from pprint import pprint
from datetime import datetime as dt
from dataclasses import dataclass
from collections import namedtuple


payloads = [
    "ty jesteś gwiazda",
    "z tobą jest jazda",
    "chciałbym cię porwać do swego gniazda",
    "dla ciebie kawior, szampan , truskawki",
    "i jakie tylko zechcesz zabawki"]









# Fundament blockchain - Block

In [3]:

@dataclass
class Block:
    index: int
    payload: str
    prev_hash: str    
    timestamp = time.time()    

    def __repr__(self):
        return f'[Block #{self.index}, payload={self.payload}, ' \
               f'hash={self.hash()}, datetime={dt.fromtimestamp(self.timestamp)}]'

    def _sha(self):
        sha = hashlib.sha256()
        sha.update(bytes(self.index))
        sha.update((str(self.timestamp) + json.dumps(self.payload) +
                    self.prev_hash).encode('utf-8'))
        return sha

    def hash(self):
        return self._sha().hexdigest()


# Blockchain

In [4]:
class Blockchain(object):
    def __init__(self):
        self.index = 0
        self.blocks = []
        self.last_hash = ''

    def __repr__(self):
        return f'[Blockchain, height={len(self.blocks)}, ' \
               f'last 5 blocks={str(self.blocks[-5:])}]'

    def add_block(self, payload):
        block = Block(self.index, payload, self.last_hash)
        self.blocks.append(block)
        self.index += 1
        self.last_hash = block.hash()
        return block
    
    

In [5]:
blockchain = Blockchain()
blockchain.add_block('w ogóle centralnie kamieniem go bez kitu')


[Block #0, payload=w ogóle centralnie kamieniem go bez kitu, hash=e8e94e24c485a2f3d155f3f133e0fe58b5df6a27eccce0887251dbe165795d68, datetime=2018-09-18 17:50:34.176115]

# Dodajmy kilka bloków

In [6]:
for i in range(10):
    payload = f"[{i}] {random.choice(payloads)}"
    block = blockchain.add_block(payload)
    print(f"Block added:\n \t {block}")

Block added:
 	 [Block #1, payload=[0] z tobą jest jazda, hash=b7bf6b75c934d742cc3db4e97ae1ac21c02538213fb4ace2254468c6da0a8890, datetime=2018-09-18 17:50:34.176115]
Block added:
 	 [Block #2, payload=[1] ty jesteś gwiazda, hash=8c11b48218defa97c273105b209d768595ed290509ff54d854e3ba10b1ecd454, datetime=2018-09-18 17:50:34.176115]
Block added:
 	 [Block #3, payload=[2] z tobą jest jazda, hash=d29914dc2fefe504123c6cd31b9c9b0a3634c4865f0023b467fd6662072d2823, datetime=2018-09-18 17:50:34.176115]
Block added:
 	 [Block #4, payload=[3] z tobą jest jazda, hash=f777b2b99ad9f8e8c67ad49884cccecb8ea2d26fefbbfe1a85683c2828979a7c, datetime=2018-09-18 17:50:34.176115]
Block added:
 	 [Block #5, payload=[4] z tobą jest jazda, hash=c3d69b416e6ef1bd275bb276f139078ba8f01d189998bf40e119b41961cf885c, datetime=2018-09-18 17:50:34.176115]
Block added:
 	 [Block #6, payload=[5] chciałbym cię porwać do swego gniazda, hash=1313c431f2203c37e89f1982d170c9e32e0f50ce341b66014d0be7108e746df4, datetime=2018-09-18 1

# Weryfikacja spójności danych

In [8]:
def verify(blockchain):
    for block, prev_block in zip(blockchain.blocks[1:], blockchain.blocks):
        if block.prev_hash != prev_block.hash():
            print(f'!!! Blockchain verification FAILED on block #{block.index} - {block.payload} !!!')
            break
    else:
        print('Blockchain verification PASSED')

## Weryfikacja poprawnych danych

In [9]:
verify(blockchain)

Blockchain verification PASSED


## Weryfikacja po drobnej zmianie w bloku

In [10]:
blockchain.blocks[3].payload += " Jam jest 444"
print(blockchain.blocks[3].payload)


[2] z tobą jest jazda Jam jest 444


In [12]:
verify(blockchain)

!!! Blockchain verification FAILED on block #4 - [3] z tobą jest jazda !!!


# Teraz na poważnie - transakcje


## Definicja transakcji

In [13]:
Transaction = namedtuple('Transaction', 'addr_from, addr_to, amount')

In [14]:
def create_test_transactions(blockchain):
    blockchain.add_block([Transaction('1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA',
                                  '19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx',
                                  1.0)])

    blockchain.add_block([Transaction('12vi3T1DifT4y7tyz5TGbyFQwj2opmmaia',
                                      '1Gdw1B4dMTZHQUzv7ggnPmZWap9pjzyn4d',
                                      2.3), 
                          Transaction('1PTTZ2gKWBKPuC8JKtFd5sDbT4XQgEzvSL',
                                      '12FuBeX5Ruyd4UxeCeHuAABuQ9cD2aGQKo',
                                      10.0)])

    blockchain.add_block([Transaction('19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx',
                                      '1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA',
                                      1.0)])


In [15]:
serious_blockchain = Blockchain()
serious_blockchain.add_block("w ogóle centralnie kamieniem go bez kitu")

[Block #0, payload=w ogóle centralnie kamieniem go bez kitu, hash=e8e94e24c485a2f3d155f3f133e0fe58b5df6a27eccce0887251dbe165795d68, datetime=2018-09-18 17:50:34.176115]

In [16]:
create_test_transactions(serious_blockchain)

In [ ]:
serious_blockchain.blocks

In [ ]:
verify(serious_blockchain)

# Mining

In [ ]:
class Block:
    def __init__(self, index, payload, prev_hash):
        self.index = index
        self.timestamp = time.time()
        self.payload = payload
        self.prev_hash = prev_hash
        self.nonce = 0
        self.difficulty = 0

    def __repr__(self):
        return f'[Block #{self.index}, payload={self.payload}, ' \
               f'hash={self.hash()}, nonce={self.nonce}, diff={self.difficulty}, ' \
               f'datetime={dt.fromtimestamp(self.timestamp)}, ' \
               f'prev_hash={self.prev_hash}]'

    def _sha(self):
        sha = hashlib.sha256()
        sha.update(bytes(self.index) + bytes(self.nonce) + bytes(self.difficulty))
        sha.update((str(self.timestamp) + json.dumps(self.payload) + 
                    self.prev_hash).encode('utf-8'))
        return sha

    def hash(self):
        return self._sha().hexdigest()

    def difficulty_matches(self):
        bits = ''.join(f"{c:08b}" for c in self._sha().digest())
        prefix = bits[:self.difficulty]
        mask = '0' * self.difficulty
        return prefix == mask

    def mine(self, difficulty):
        self.difficulty = difficulty
        while not self.difficulty_matches():
            print("nonce", self.nonce)
            self.nonce += 1


class Blockchain(object):
    def __init__(self):
        self.index = 0
        self.last_hash = ''
        self.blocks = []
        self.difficulty = 0
        self.add_block('The Times 03/Jan/2009 Chancellor on brink of second bailout ' \
                       'for banks')

    def __repr__(self):
        return f'[Blockchain, height={len(self.blocks)}, ' \
               f'last 5 blocks={str(self.blocks[-5:])}]'

    def add_block(self, payload):
        block = Block(self.index, payload, self.last_hash)
        block.mine(self.difficulty)
        self.blocks.append(block)
        self.index += 1
        self.last_hash = block.hash()
        return block
    
    def verify(self):
        for block, prev_block in zip(self.blocks[1:], self.blocks):
            if block.prev_hash != prev_block.hash():
                print(f'Verification FAILED on block #{block.index}')
                break
        else:
            print('Verification PASSED')

In [ ]:
hard_blockchain = Blockchain()

In [ ]:
%%time
create_test_transactions(hard_blockchain)

In [ ]:
hard_blockchain.verify()

## Zwiększamy poziom trudności...

In [ ]:
%%time
much_harder_blockchain = Blockchain()
much_harder_blockchain.difficulty = 5  # hardcore level
create_test_transactions(much_harder_blockchain)

In [ ]:
verify(much_harder_blockchain)